Game design E20 with abilities

## Spud code

In [1]:
using Random
using DataFrames
using CSV
using DelimitedFiles
using Statistics
using Printf
using LinearAlgebra
using Dates
using JuMP
using HiGHS

In [2]:
struct Spud
    name::String
    h::Int64
    f::Int64
    l::Int64
    p::Int64
    r::Int64
    s::Int64
    a1::Int64
end

In [54]:
ability_none = 999
ability_add1 = 100
ability_your_H_their_P = 400
ability_your_F_their_L = 450
ability_your_F_their_S = 470
ability_your_L_their_S = 500
ability_your_P_their_L = 545
ability_your_P_their_R = 550
ability_your_R_their_S = 600
ability_ninja = 502
ability_eloquent = 510
ability_slow_cancel = 200

ability_name = Dict(
    ability_add1 => "add1",
    ability_your_H_their_P => "yourHtheirP",
    ability_your_F_their_L => "yourFtheirL",
    ability_your_F_their_S => "yourFtheirS",
    ability_your_L_their_S => "yourLtheirS",
    ability_your_P_their_R => "yourPtheirL",
    ability_your_P_their_R => "yourPtheirR",
    ability_your_R_their_S => "yourRtheirS",
    ability_ninja => "ninja",
    ability_none => "vanilla",
    ability_eloquent => "eloquent",
    ability_slow_cancel => "slowCancel",
)

function fx_add1(a::Spud, b::Spud)
    b0 = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.s, ability_none)
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r, a.s+1, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p, a.r+1, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l, a.p+1, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f, a.l+1, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h, a.f+1, a.l, a.p, a.r, a.s, ability_none)
    end
    if eval_battle(Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none), b0)==1
        a = Spud(a.name, a.h+1, a.f, a.l, a.p, a.r, a.s, ability_none)
    end
    return [a, b]
end

function fx_your_H_their_P(a::Spud, b::Spud)
    a = Spud(a.name, b.p, a.f, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, b.l, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_F_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, b.s, a.l, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_L_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, b.s, a.p, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_L(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, b.l, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_P_their_R(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, b.r, a.r, a.s, a.a1)
    return [a, b]
end

function fx_your_R_their_S(a::Spud, b::Spud)
    a = Spud(a.name, a.h, a.f, a.l, a.p, b.s, a.s, a.a1)
    return [a, b]
end

function fx_ninja(a::Spud, b::Spud)
    if (a.h > b.f) && (a.f > b.h)
        a = Spud(a.name, a.h, a.f, a.l+1, a.p+1, a.r+1, a.s+1, a.a1)
    end
    return [a, b]
end

function fx_eloquent(a::Spud, b::Spud)
    if (a.p > b.p) && (a.r > b.r) && (a.s > b.s)
        a = Spud(a.name, a.h, a.f, a.l+6, a.p, a.r, a.s, a.a1)
    end
    return [a, b]
end

function fx_slow_cancel(a::Spud, b::Spud)
    b = Spud(b.name, b.h, b.f, b.l, b.p, b.r, b.s, ability_none)    
    return [a, b]
end


ability_fx = Dict(
    ability_ninja => fx_ninja,
    ability_add1 => fx_add1,
    ability_your_H_their_P => fx_your_H_their_P,
    ability_your_F_their_L => fx_your_F_their_L,
    ability_your_F_their_S => fx_your_F_their_S,
    ability_your_L_their_S => fx_your_L_their_S,
    ability_your_P_their_L => fx_your_P_their_L,
    ability_your_P_their_R => fx_your_P_their_R,
    ability_your_R_their_S => fx_your_R_their_S,
    ability_eloquent => fx_eloquent,
    ability_slow_cancel => fx_slow_cancel,
)

Dict{Int64, Function} with 11 entries:
  545 => fx_your_P_their_L
  200 => fx_slow_cancel
  550 => fx_your_P_their_R
  450 => fx_your_F_their_L
  510 => fx_eloquent
  470 => fx_your_F_their_S
  502 => fx_ninja
  500 => fx_your_L_their_S
  600 => fx_your_R_their_S
  400 => fx_your_H_their_P
  100 => fx_add1

In [4]:
#const MXS = 100
const MXV = 9
const MNV = 1

# function cost(h::Int64, f::Int64, l::Int64, p::Int64, r::Int64, s::Int64)::Int64
#     #return h*(f+l-2*MNV) + f*(p+r+s-3*MNV) + h+f+l+p+r+s-6*MNV
#     return h*(f+l) + f*(p+r+s)
# end

# function cost(a::Spud)::Int64
#     if a.a1 == ability_none
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s)
#     else
#         return cost(a.h, a.f, a.l, a.p, a.r, a.s) + ability_cost[a.a1](a)
#     end
# end

1

In [5]:
function compare_int_list(as::Vector{Int64}, bs::Vector{Int64}, tiebreaker::Int64 = 0)::Int64
    n = min(length(as), length(bs))
    for i in 1:n
        if as[i] != bs[i]
            return sign(as[i] - bs[i])
        end
    end
    return tiebreaker
end

function spud_h_seq(a::Spud)::Vector{Int64}
    return [a.h, a.s, a.r, a.p, a.l, a.f]
end

function spud_f_seq(a::Spud)::Vector{Int64}
    return [a.f, a.s, a.r, a.p, a.l, a.h]
end

function spud_l_seq(a::Spud)::Vector{Int64}
    return [a.l]
end

function spud_p_seq(a::Spud)::Vector{Int64}
    return [a.p, a.l]
end

function spud_r_seq(a::Spud)::Vector{Int64}
    return [a.r, a.f]
end

function spud_s_seq(a::Spud)::Vector{Int64}
    return [a.s, a.h]
end

function spud_utb_seq(a::Spud)::Vector{Int64}
    return [a.a1, a.h, a.f, a.l, a.p, a.r, a.s]
end

function eval_finds(a::Spud, b::Spud, tiebreaker::Int64 = 0)::Int64
    ev = compare_int_list(spud_f_seq(a), spud_h_seq(b), tiebreaker)
    ans = ev
end

function eval_melee(a::Spud, b::Spud, tiebreaker1::Int64 = 0, tiebreaker2::Int64 = 0)::Int64
    comp_p = compare_int_list(spud_p_seq(a), spud_p_seq(b), tiebreaker1)
    comp_r = compare_int_list(spud_r_seq(a), spud_r_seq(b), tiebreaker1)
    comp_s = compare_int_list(spud_s_seq(a), spud_s_seq(b), tiebreaker1)
    ev = 4 * comp_p + 3 * comp_r + 2 * comp_s
    return sign(ev + (1-abs(ev))*tiebreaker2)
end



eval_melee (generic function with 3 methods)

In [6]:


function apply_ability(a::Spud, b::Spud)::Array{Spud}
    if a.a1 == ability_none
        return [a, b]
    else
        return ability_fx[a.a1](a, b)
    end
end


function eval_battle(a::Spud, b::Spud)::Int64
    utb = compare_int_list(spud_utb_seq(a), spud_utb_seq(b), 0) # universal tiebreaker
    if utb == 0
        return 0
    end
    a_ability_first = compare_int_list([a.a1, a.s], [b.a1, b.s], utb)
    if a_ability_first ==1
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
    end
    if a_ability_first == -1
        if b.a1 != ability_none
            res = apply_ability(b, a)
            b = res[1]
            a = res[2]
        end
        if a.a1 != ability_none
            res = apply_ability(a, b)
            a = res[1]
            b = res[2]
        end
    end
    #println(a)
    #println(b)
    a_finds = eval_finds(a, b, utb)==1
    b_finds = eval_finds(b, a, utb)==1
    melee_win = eval_melee(a, b, 0, 0)
    luck_win = compare_int_list([a.l, melee_win], [b.l, -melee_win], 0)
    if melee_win ==1 && luck_win ==1
        return 1
    end
    if melee_win == -1 && luck_win == -1
        return -1
    end
    if a_finds && b_finds
        return melee_win
    end
    if a_finds && !b_finds
        return 1
    end
    if !a_finds && b_finds
        return -1
    end
    if !a_finds && !b_finds
        return luck_win
    end
end

eval_battle (generic function with 1 method)

In [7]:
function eval_battle_list(a::Spud, bs::Array{Spud})::Int
    score = 0
    for ii in 1:length(bs)
        score = score + eval_battle(a, bs[ii])
    end
    return score
end

function eval_battle_list2(a::Spud, bs::Array{Spud}, w::Vector{Float64})::AbstractFloat
    score = 0.0
    for ii in 1:length(bs)
        score = score + w[ii] * eval_battle(a, bs[ii])
    end
    return score
end

function spuds_to_df(as::Array{Spud})::DataFrame
    names = Array{String}(undef, length(as))
    hs = Array{Int}(undef, length(as))
    fs = Array{Int}(undef, length(as))
    ls = Array{Int}(undef, length(as))
    ps = Array{Int}(undef, length(as))
    rs = Array{Int}(undef, length(as))
    ss = Array{Int}(undef, length(as))
    a1s = Array{Int}(undef, length(as))
    for ii in 1:length(as)
        names[ii] = as[ii].name
        hs[ii] = as[ii].h
        fs[ii] = as[ii].f
        ls[ii] = as[ii].l
        ps[ii] = as[ii].p
        rs[ii] = as[ii].r
        ss[ii] = as[ii].s
        a1s[ii] = as[ii].a1
    end
    df = DataFrame(name = names, h = hs, f = fs, l = ls, p = ps, r = rs, s = ss, a1 = a1s)
    return df
end

function fpart(x::AbstractFloat)::AbstractFloat
  return x - trunc(x)
end

# for legacy dfs without abilities
function df_to_spuds0(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], ability_none)
    end
    return as
end

function df_to_spuds(df::DataFrame)::Array{Spud}
    n = size(df)[1]
    as = Array{Spud}(undef, n)
    for i in 1:n
        as[i] = Spud(df[i, :name], df[i, :h], df[i, :f], df[i, :l], df[i, :p], df[i, :r], df[i, :s], df[i, :a1])
    end
    return as
end



df_to_spuds (generic function with 1 method)

## Get dishes with cost 20 and abilities

In [22]:
function get_library(cost, ab=999, n_init = 10000)
    ffs = Array{Spud}(undef, n_init)
    ff_i = 0
    hrange = MNV:MXV
    frange = MNV:MXV
    lrange = MNV:MXV
    prange = MNV:MXV
    rrange = MNV:MXV
    srange = MNV:MXV
    for h in hrange
        for f in frange
            for l in lrange
                for p in prange
                    for r in rrange
                        for s in srange
                            if (h+f+l+p+r+s == cost)
                                ff_i += 1
                                ff = Spud(string(ability_name[ab], "-", h,f,l,p,r,s), h,f,l,p,r,s,ab)
                                ffs[ff_i] = ff
                            end
                        end
                    end                        
                end                        
            end
        end
    end
    ffs = ffs[1:ff_i]
    return unique(ffs)
end

get_library (generic function with 3 methods)

In [43]:
function get_payoffs(env::Array{Spud})::Array{Int64}
    n_nash = length(env)
    payoffs = Array{Int64}(undef, (n_nash, n_nash))
    for i in 1:n_nash
        for j in 1:n_nash
            payoffs[i, j] = eval_battle(env[i], env[j])
        end
    end
    return payoffs
end

function spud2int(a::Spud)::Int64
    return a.h * 100000 + a.f * 10000 + a.l * 1000 + a.p * 100 + a.r * 10 + a.s
end

spud2int (generic function with 1 method)

In [34]:
function get_nash_env(lib::Array{Spud})::Tuple{Array{Spud}, Array{Float64}}
    ## optimize
    payoffs = get_payoffs(lib);
    vector_model = Model(HiGHS.Optimizer)
    n_nash = length(lib)
    sumv = zeros(Int64, (1, 2 * n_nash + 1))
    sumv[1:n_nash] .= 1
    A = hcat(payoffs, Matrix(1I,n_nash, n_nash), -ones(Int64, n_nash, 1))
    A = vcat(A, sumv);
    b = zeros(Int64, n_nash + 1);
    b[1:n_nash] .= -1
    b[n_nash+1] = 1
    transpose(b);
    c = zeros(Int64, 2 * n_nash + 1);
    c[2 * n_nash + 1] = 1;
    @variable(vector_model, x[1:(2*n_nash+1)] >= 0);
    @constraint(vector_model, A * x .== b);
    @objective(vector_model, Min, c' * x)
    @time optimize!(vector_model)
    ## solution
    w_lp = [value(x[i]) for i in 1:n_nash]
    w_lp = w_lp./sum(w_lp);
    new_env = lib[w_lp .> 0]
    w_new = w_lp[w_lp .> 0];
    for i in sortperm(-w_new)
        print(new_env[i])
        print(" ")
        println(string(w_new[i])[1:6])
    end
    return new_env, w_new
end

get_nash_env (generic function with 1 method)

In [55]:
lib = vcat(
    get_library(20, ability_none, 20000),
    get_library(19, ability_ninja, 20000),
    get_library(19, ability_slow_cancel, 20000),
    get_library(19, ability_eloquent, 20000),
    get_library(17, ability_add1, 20000)
)
length(lib)

37794

In [56]:
sublib = unique([rand(lib) for i in 1:1000])
ne, wn = get_nash_env(sublib);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
987 rows, 1973 cols, 974130 nonzeros
987 rows, 1970 cols, 971176 nonzeros
Presolve : Reductions: rows 987(-0); columns 1970(-3); elements 971176(-2954)
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 987(987) 0s
       1607     1.0000000000e+00 Pr: 0(0) 0s
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Simplex   iterations: 1607
Objective value     :  1.0000000000e+00
HiGHS run time      :          0.73
  0.759005 seconds (13.04 k allocations: 57.528 MiB)
Spud("vanilla-714215", 7, 1, 4, 2, 1, 5, 999) 0.0777
Spud("eloquent-517141", 5, 1, 7, 1, 4, 1, 510) 0.0775
Spud("slowCancel-627121", 6, 2, 7, 1, 2, 1, 200) 0.0642
Spud("vanilla-141671", 1, 4, 1, 6, 7, 1, 999) 0.0609
Spud("eloquent-151453", 1, 5, 1, 4, 5, 3, 510) 0.0526
Spud("eloquent-123445", 1, 2, 3

In [62]:
scores = [eval_battle_list2(ff, ne, wn) for ff in lib];
sublib = lib[sortperm(-scores)[1:1000]]
ne, wn = get_nash_env(sublib);

Running HiGHS 1.6.0: Copyright (c) 2023 HiGHS under MIT licence terms
Presolving model
1001 rows, 2001 cols, 1001990 nonzeros
1001 rows, 2001 cols, 1001990 nonzeros
Presolve : Reductions: rows 1001(-0); columns 2001(-0); elements 1001990(-0) - Not reduced
Problem not reduced by presolve: solving the LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0     0.0000000000e+00 Pr: 1001(1001) 0s
       2600     1.0000000000e+00 Pr: 0(0) 1s
       2600     1.0000000000e+00 Pr: 0(0) 1s
Model   status      : Optimal
Simplex   iterations: 2600
Objective value     :  1.0000000000e+00
HiGHS run time      :          1.91
  1.956091 seconds (13.22 k allocations: 59.250 MiB, 0.46% gc time)
Spud("add1-419111", 4, 1, 9, 1, 1, 1, 100) 0.0323
Spud("add1-319112", 3, 1, 9, 1, 1, 2, 100) 0.0201
Spud("vanilla-131816", 1, 3, 1, 8, 1, 6, 999) 0.0175
Spud("vanilla-718211", 7, 1, 8, 2, 1, 1, 999) 0.0164
Spud("eloquent-114643", 1, 1, 4, 6, 4, 3, 510) 0.01

Spud("vanilla-161714", 1, 6, 1, 7, 1, 4, 999) 0.0015
Spud("ninja-271522", 2, 7, 1, 5, 2, 2, 502) 0.0015
Spud("vanilla-121871", 1, 2, 1, 8, 7, 1, 999) 0.0015
Spud("eloquent-113482", 1, 1, 3, 4, 8, 2, 510) 0.0015
Spud("vanilla-716132", 7, 1, 6, 1, 3, 2, 999) 0.0014
Spud("slowCancel-219124", 2, 1, 9, 1, 2, 4, 200) 0.0014
Spud("eloquent-114274", 1, 1, 4, 2, 7, 4, 510) 0.0014
Spud("eloquent-114337", 1, 1, 4, 3, 3, 7, 510) 0.0014
Spud("eloquent-113437", 1, 1, 3, 4, 3, 7, 510) 0.0014
Spud("vanilla-718121", 7, 1, 8, 1, 2, 1, 999) 0.0013
Spud("vanilla-319142", 3, 1, 9, 1, 4, 2, 999) 0.0013
Spud("vanilla-121268", 1, 2, 1, 2, 6, 8, 999) 0.0013
Spud("vanilla-219314", 2, 1, 9, 3, 1, 4, 999) 0.0013
Spud("add1-141263", 1, 4, 1, 2, 6, 3, 100) 0.0013
Spud("vanilla-815411", 8, 1, 5, 4, 1, 1, 999) 0.0012
Spud("eloquent-113464", 1, 1, 3, 4, 6, 4, 510) 0.0012
Spud("eloquent-315424", 3, 1, 5, 4, 2, 4, 510) 0.0012
Spud("add1-161216", 1, 6, 1, 2, 1, 6, 100) 0.0012
Spud("vanilla-319214", 3, 1, 9, 2, 1, 4, 999)

In [61]:
maximum(scores), sum(scores .> 0)

(0.0751971134381181, 176)

In [63]:
df = spuds_to_df(ne)
df[!, :w] = [floor(w*1000)/1000 for w in wn]
print(df[sortperm(-wn),:])

259×9 DataFrame
 Row │ name               h      f      l      p      r      s      a1     w       
     │ String             Int64  Int64  Int64  Int64  Int64  Int64  Int64  Float64 
─────┼─────────────────────────────────────────────────────────────────────────────
   1 │ add1-419111            4      1      9      1      1      1    100    0.032
   2 │ add1-319112            3      1      9      1      1      2    100    0.02
   3 │ vanilla-131816         1      3      1      8      1      6    999    0.017
   4 │ vanilla-718211         7      1      8      2      1      1    999    0.016
   5 │ eloquent-114643        1      1      4      6      4      3    510    0.015
   6 │ vanilla-818111         8      1      8      1      1      1    999    0.013
   7 │ vanilla-151751         1      5      1      7      5      1    999    0.013
   8 │ vanilla-619211         6      1      9      2      1      1    999    0.013
   9 │ slowCancel-419311      4      1      9      3      1      1   

In [64]:
lib = ne
payoffs = get_payoffs(lib)
nlib = length(lib)
combos = Array{Spud}(undef, (10000,3))
#ord = sortperm([rand() for i in 1:nlib])[1:800]
ord = [i for i in 1:nlib]
#ord = [i for i in 1:(126*4)]
nc = 0
# 3-combos
for i in ord[1:(length(ord)-2)]
    print(i)
    print(" ")
    pi = payoffs[i,:]
    for j in ord[(i+1):(length(ord)-1)]
        pj = payoffs[j, :]
        pij = [max(pi[a], pj[a]) for a in 1:nlib]
        if sum(pij .== 1) > 0
            #print(".")
            for k in ord[(j+1):length(ord)]
                pk = payoffs[k, :]
                pijk = [max(pi[a], pj[a], pk[a]) for a in 1:nlib]
                if minimum(pijk)==1
                    nc += 1
                    combos[nc, :] = lib[[i,j,k]]
                    #println("")
                    #println("----3-COMBO!-----")
                    #println(lib[[i,j,k]])
                end
            end
        end
    end
end
combos = combos[1:nc,:];

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 

In [65]:
size(combos)

(286, 3)

### combo-breaker research

In [66]:
as = combos[1,:]
print(spuds_to_df(as))

3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ String          Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────
   1 │ add1-122147         1      2      2      1      4      7    100
   2 │ vanilla-171614      1      7      1      6      1      4    999
   3 │ vanilla-718211      7      1      8      2      1      1    999

In [67]:
println(maximum([eval_battle_list(ff, as) for ff in lib]))
b = Spud("breaker",1,2,4,1,1,4,ability_your_H_their_P)
println([eval_battle(b, ff) for ff in as])
eval_battle_list2(b, ne, wn)

1
[-1, -1, -1]


-0.6277869432423939

In [70]:
for i in 1:20
    as = combos[rand(1:size(combos)[1]),:]
    println(spuds_to_df(as))
end

3×8 DataFrame
 Row │ name               h      f      l      p      r      s      a1    
     │ String             Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼────────────────────────────────────────────────────────────────────
   1 │ vanilla-718121         7      1      8      1      2      1    999
   2 │ vanilla-171155         1      7      1      1      5      5    999
   3 │ slowCancel-121717      1      2      1      7      1      7    200
3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ String          Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────
   1 │ ninja-281431        2      8      1      4      3      1    502
   2 │ vanilla-112718      1      1      2      7      1      8    999
   3 │ vanilla-616115      6      1      6      1      1      5    999
3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ Str

3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ String          Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────
   1 │ add1-151514         1      5      1      5      1      4    100
   2 │ vanilla-121268      1      2      1      2      6      8    999
   3 │ vanilla-519122      5      1      9      1      2      2    999
3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ String          Int64  Int64  Int64  Int64  Int64  Int64  Int64 
─────┼─────────────────────────────────────────────────────────────────
   1 │ vanilla-121268      1      2      1      2      6      8    999
   2 │ vanilla-619121      6      1      9      1      2      1    999
   3 │ vanilla-161615      1      6      1      6      1      5    999
3×8 DataFrame
 Row │ name            h      f      l      p      r      s      a1    
     │ String          Int64